<a href="https://colab.research.google.com/github/gevenbly/TensorAlgs/blob/main/test_network_solve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# uncomment code below if running locally

# !git clone https://github.com/gevenbly/TensorAlgs
# import os
# os.chdir('/content/TensorAlgs')

In [ ]:
import numpy as np
from typing import Optional, List, Union, Tuple
from network_solve import (
    call_solver, ord_to_ncon, ncon_to_weighted_adj, full_solve_complete,
    greedy_cost_solve, greedy_size_solve)

In [ ]:
"""
Unit tests for 'network_solve'

- call_solver
- ord_to_ncon
- ncon_to_weighted_adj
- full_solve_complete
- greedy_cost_solve
- greedy_size_solve
"""

In [ ]:
# test call_solver
chi = np.random.randint(2, 10)
u = np.random.rand(chi, chi, chi, chi)
w = np.random.rand(chi, chi, chi)
ham = np.random.rand(chi, chi, chi, chi, chi, chi)
tensors = [u, u, w, w, w, ham, u, u, w, w, w]
connects = [[1, 3, 10, 11], [4, 7, 12, 13], [8, 10, -4], [11, 12, -5],
            [13, 14, -6], [2, 5, 6, 3, 4, 7], [1, 2, 9, 17], [5, 6, 16, 15],
            [8, 9, -1], [17, 16, -2], [15, 14, -3]]
con_order, costs, is_optimal = call_solver(tensors, connects, max_branch=None)

flat_connects = np.concatenate(connects)
inds = np.sort(np.unique(flat_connects[flat_connects > 0]))
ex_cost = np.log10(2 * chi**9 + 4 * chi**8 + 2 * chi**6 + 2 * chi**5)
assert np.allclose(costs, ex_cost)
assert is_optimal
assert np.array_equal(inds, np.sort(con_order))

In [ ]:
# test ncon_to_weighted_adj
N = 5
chi = 4
A = np.zeros([chi, chi])
tensors = [A] * N
dims = [tensor.shape for tensor in tensors]
connects = [0] * N
for k in range(N):
  if k == 0:
    connects[k] = [-1, 1]
  elif k == (N - 1):
    connects[k] = [k, -2]
  else:
    connects[k] = [k, k + 1]
log_adj = ncon_to_weighted_adj(dims, connects)
ex_log_adj = np.zeros([N, N])
ex_log_adj[:(N - 1), 1:] = np.diag(np.log10(chi) * np.ones([N - 1]))
ex_log_adj += ex_log_adj.T
ex_log_adj[0, 0] = np.log10(chi)
ex_log_adj[-1, -1] = np.log10(chi)
assert np.allclose(log_adj, ex_log_adj)

In [ ]:
# test ord_to_ncon
N = 8
num_closed = 8
num_open = 4 * N - 2 * num_closed
cl_inds = 1 + np.arange(num_closed)
op_inds = -1 - np.arange(num_open)
connects = [0] * N
perm = np.argsort(np.random.rand(4 * N))
comb_inds = np.concatenate((op_inds, cl_inds, cl_inds))[perm]
for k in range(N):
  if k < (N - 1):
    connect_temp = np.concatenate((comb_inds[4 * k:4 * (k + 1)],
                                    [num_closed + k + 1, num_closed + k + 2]))
  else:
    connect_temp = np.concatenate(
        (comb_inds[4 * k:4 * (k + 1)], [num_closed + k + 1, num_closed + 1]))
  connects[k] = list(connect_temp[np.argsort(np.random.rand(6))])
order = np.zeros([2, N - 1], dtype=int)
for k in range(N - 1):
  temp_loc = np.random.randint(0, N - k - 1)
  order[0, k] = temp_loc
  order[1, k] = np.random.randint(temp_loc + 1, N - k)
con_order = ord_to_ncon(connects, order)
assert np.array_equal(np.sort(con_order), np.arange(num_closed + N) + 1)

In [ ]:
# test greedy_size_solve
N = 10
log_adj = (1 + np.sin(range(N**2))).reshape(N, N)
log_adj += log_adj.T
order, cost = greedy_size_solve(log_adj)
assert order.shape == (2, N - 1)
assert isinstance(cost, float)

d1 = 6
d2 = 8
N = 3
log_adj = np.zeros([N, N])
log_adj[0, 1] = d1
log_adj[1, 2] = d2
log_adj += log_adj.T
order, cost = greedy_size_solve(log_adj)
if d1 >= d2:
  ex_order = np.array([[0, 0], [1, 1]])
  ex_cost = d2 + np.log10(10**d1 + 1)
else:
  ex_order = np.array([[1, 0], [2, 1]])
  ex_cost = d1 + np.log10(10**d2 + 1)
assert np.array_equal(order, ex_order)
assert np.allclose(ex_cost, cost)

In [ ]:
# test greedy_cost_solve
N = 12
log_adj = (1 + np.sin(range(N**2))).reshape(N, N)
log_adj += log_adj.T
order, cost = greedy_cost_solve(log_adj)
assert order.shape == (2, N - 1)
assert isinstance(cost, float)

d1 = 5
d2 = 4
d3 = 3
N = 3
log_adj = np.zeros([N, N])
log_adj[0, 1] = d1
log_adj[1, 2] = d2
log_adj += log_adj.T
log_adj[2, 2] = d3
order, cost = greedy_cost_solve(log_adj)
ex_order = np.array([[0, 0], [1, 1]])
ex_cost = d1 + d2 + np.log10(1 + 10**(d3 - d1))
assert np.array_equal(order, ex_order)
assert np.allclose(ex_cost, cost)

In [ ]:
# test full_solve_complete
N = 6
log_adj = (1 + np.sin(range(N**2))).reshape(N, N)
log_adj += log_adj.T
order, cost, _ = full_solve_complete(log_adj)
assert order.shape == (2, N - 1)
assert isinstance(cost, float)

d1 = 6
N = 7
log_adj = np.zeros([N, N])
log_adj[:(N - 1), 1:] = np.diag(d1 * np.ones(N - 1))
log_adj += log_adj.T
log_adj[0, 0] = d1
log_adj[-1, -1] = d1
_, cost, is_optimal = full_solve_complete(log_adj)
ex_cost = np.log10((N - 1) * 10**(3 * d1))
assert np.allclose(ex_cost, cost)
assert is_optimal

cost_bound = 50
max_branch = 500
N = 7
log_adj = (1 + np.sin(range(N**2))).reshape(N, N)
log_adj += log_adj.T
order, cost, _ = full_solve_complete(
    log_adj, cost_bound=cost_bound, max_branch=max_branch)
assert order.shape == (2, N - 1)
assert isinstance(cost, float)